In [52]:
import easyocr
from PIL import Image
import openai
import os
from dotenv import load_dotenv
import requests
import re

In [53]:
load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

if not OPENROUTER_API_KEY:
    print("Error: OPENROUTER_API_KEY not found in .env file.")
else:
    print("API Key loaded successfully.")

API Key loaded successfully.


In [54]:
def clean_ocr_text(text):
    # ลบช่องว่างเกิน และแทนที่ด้วยช่องว่างเดียว
    text = re.sub(r'\s+', ' ', text)
    
    # แก้คำผิดที่พบบ่อยจาก OCR ด้วย Dictionary
    common_errors = {
        "จาย": "จ่าย",
        "เงืน": "เงิน",
        "ณฺ": "ณ",
        "ฺณ": "ณ",
        "ค้างจาย": "ค้างจ่าย",
        "รวมจำนวนเงืน": "รวมจำนวนเงิน",
        "รวมเป็นเง็บ": "รวมเป็นเงิน",
        "รามภาษี": "รวมภาษี",
        "ราคารวม": "รวมราคา",
        "จำนวบ": "จำนวน",
        "หนวยบับ": "หน่วยนับ",
    }
    for wrong, correct in common_errors.items():
        text = text.replace(wrong, correct)
    
    # แก้ตัวอักษรที่ OCR สับสน เช่น d, q, o ให้เป็น 0 (zero)
    text = text.replace('d', '0').replace('q', '0').replace('o', '0')
    
    return text

sample_text = "2 20d,00"
print("Before:", sample_text)
print("After: ", clean_ocr_text(sample_text))

Before: 2 20d,00
After:  2 200,00


In [62]:
image_path = "../Back-end/sample/Test5.png"  

Image.open(image_path).show()

reader = easyocr.Reader(['th', 'en'])
ocr_result = reader.readtext(image_path, detail=0)  

raw_text = "\n".join(ocr_result)
print("📄 ข้อความจาก OCR (ดิบ):\n", raw_text)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


📄 ข้อความจาก OCR (ดิบ):
 บัญชี
จำนวนเงิน
0201001.2200000001.680101010000022.5080101000001.000000000.682220232600120.02
42,500.00
กองทุนเพื่อการศึกษา.วิทยาลัยศิลปะ สื่อ และเทคโนโลยี.งานจัดการศึกษาสาขาศิลปะ เสื่อ และเทคโนโลยี
 งบปี
2568.ศาจาง
-จายจากเงินอุดหนุนทั่วไป.ไมระบุ.รด68-d2innovative learning platform
นวัตกรรมการจัดการเรียนการสอนรูปแบบใหม่/งานจัดการศึกษาสาขาศิลปะ  เสื่อ
และเทคโนโลยี/อุดหนุนทั่วไป-อุดหนุนโครงการการสร้างพื้นที่การเรียนรู้สำหรับนักศึกษา(learning playground )..เงินรายได้
รวมจำนวนเงืน
42,500,00
 จำนวนเงินที่นำมาคำนวณภาษีหักฺณ ที่จ่าย
42,500.00
ภาษีเงินได้หักฺณที่จายค้างจาย
% เงืนรายได
425.00
 จำนวนเงินสุทธิหลังภาษี ณฺที่จ่าย
42,075.00


In [63]:
clean_text = clean_ocr_text(raw_text)
print("📄 ข้อความหลังแก้ไข OCR:\n", clean_text)

📄 ข้อความหลังแก้ไข OCR:
 บัญชี จำนวนเงิน 0201001.2200000001.680101010000022.5080101000001.000000000.682220232600120.02 42,500.00 กองทุนเพื่อการศึกษา.วิทยาลัยศิลปะ สื่อ และเทคโนโลยี.งานจัดการศึกษาสาขาศิลปะ เสื่อ และเทคโนโลยี งบปี 2568.ศาจาง -จ่ายจากเงินอุดหนุนทั่วไป.ไมระบุ.รด68-02inn0vative learning platf0rm นวัตกรรมการจัดการเรียนการสอนรูปแบบใหม่/งานจัดการศึกษาสาขาศิลปะ เสื่อ และเทคโนโลยี/อุดหนุนทั่วไป-อุดหนุนโครงการการสร้างพื้นที่การเรียนรู้สำหรับนักศึกษา(learning playgr0un0 )..เงินรายได้ รวมจำนวนเงิน 42,500,00 จำนวนเงินที่นำมาคำนวณภาษีหักณ ที่จ่าย 42,500.00 ภาษีเงินได้หักณที่จ่ายค้างจ่าย % เงินรายได 425.00 จำนวนเงินสุทธิหลังภาษี ณที่จ่าย 42,075.00


In [64]:
user_prompt = """
📌 [THAI VERSION BELOW]

จากข้อความ OCR ด้านล่างนี้ กรุณาดึงเฉพาะยอดรวมสุทธิที่พิมพ์ไว้ในเอกสาร โดยต้องเป็นยอดรวมที่เอกสารคำนวณไว้แล้วอย่างชัดเจน

✅ ยอดที่ต้องการ:
- ยอดรวมสุทธิที่รวมภาษีมูลค่าเพิ่มแล้ว (ถ้ามี)
- มักอยู่ใต้คำว่า “รวมสุทธิ”, “รวมทั้งสิ้น”, “จำนวนเงินสุทธิ”, “จำนวนเงินสุทธิหลังภาษี ณที่จ่าย” หรือคำในภาษาอังกฤษ เช่น “Total”, “Final Amount”, “Grand Total”, “Balance Due”, “Amount Due”, “Balance Owing”
- หากพบข้อความที่ชัดเจน เช่น "จำนวนเงินสุทธิหลังภาษี ณที่จ่าย 42,075.00" ให้ตอบกลับเป็น "42,075.00"
- ต้องเป็นยอดที่พิมพ์ไว้แล้ว ไม่ใช่คำนวณเอง

❌ ห้าม:
- ห้ามรวมจากรายการในตารางหรือคำนวณ unit price × quantity เอง
- ห้ามใช้เลขจากหมายเหตุหรือคำอธิบายท้ายเอกสาร
- ห้ามเดาจากตัวเลขที่อ่านได้โดยไม่มีความชัดเจน

หากไม่พบยอดรวมที่ชัดเจนในเอกสารนี้ ให้ตอบว่า: **ไม่พบยอดรวมที่ชัดเจนในเอกสารนี้**

📌 [ENGLISH VERSION]

From the OCR text below, extract only the final total amount that is clearly printed in the document.

✅ What we want:
- The final total amount that includes tax (if applicable) and is explicitly printed.
- It may appear under labels like "Total", "Final Amount", "Grand Total", "Net Amount", "Balance Due", "Amount Due", or "Balance Owing".
- If the printed number has minor OCR errors (e.g., '2 20d,00'), correct them logically.

❌ Do NOT:
- Do not calculate totals from individual rows (e.g., do not multiply unit price × quantity).
- Do not use numbers from comments, footnotes, or non-summary text.
- Do not infer totals if they are not clearly printed.

If no clear total is found, reply: **Total amount not clearly stated in the document**
"""

prompt = f"""
คุณคือผู้ช่วยอ่านข้อความจากใบเสร็จหรือเอกสารทางการเงิน

ข้อความที่ได้จาก OCR คือ:

{text}

{user_prompt}
"""
print("📝 Prompt ที่จะส่งให้ GPT:\n", prompt)

📝 Prompt ที่จะส่งให้ GPT:
 
คุณคือผู้ช่วยอ่านข้อความจากใบเสร็จหรือเอกสารทางการเงิน

ข้อความที่ได้จาก OCR คือ:

บัญชี
จำนวนเงิน
0201001.2200000001.680101010000022.5080101000001.000000000.682220232600120.02
42,500.00
กองทุนเพื่อการศึกษา.วิทยาลัยศิลปะ สื่อ และเทคโนโลยี.งานจัดการศึกษาสาขาศิลปะ เสื่อ และเทคโนโลยี
 งบปี
2568.ศาจาง
-จายจากเงินอุดหนุนทั่วไป.ไมระบุ.รด68-d2innovative learning platform
นวัตกรรมการจัดการเรียนการสอนรูปแบบใหม่/งานจัดการศึกษาสาขาศิลปะ  เสื่อ
และเทคโนโลยี/อุดหนุนทั่วไป-อุดหนุนโครงการการสร้างพื้นที่การเรียนรู้สำหรับนักศึกษา(learning playground )..เงินรายได้
รวมจำนวนเงืน
42,500,00
 จำนวนเงินที่นำมาคำนวณภาษีหักฺณ ที่จ่าย
42,500.00
ภาษีเงินได้หักฺณที่จายค้างจาย
% เงืนรายได
425.00
 จำนวนเงินสุทธิหลังภาษี ณฺที่จ่าย
42,075.00


📌 [THAI VERSION BELOW]

จากข้อความ OCR ด้านล่างนี้ กรุณาดึงเฉพาะยอดรวมสุทธิที่พิมพ์ไว้ในเอกสาร โดยต้องเป็นยอดรวมที่เอกสารคำนวณไว้แล้วอย่างชัดเจน

✅ ยอดที่ต้องการ:
- ยอดรวมสุทธิที่รวมภาษีมูลค่าเพิ่มแล้ว (ถ้ามี)
- มักอยู่ใต้คำว่า “รวมสุทธิ”, “รวมทั้งสิ้

In [65]:
headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "HTTP-Referer": "https://localhost", 
    "X-Title": "ReadReciptGPT"
}

data = {
    "model": "openai/gpt-3.5-turbo",
    "messages": [
        {"role": "user", "content": prompt}
    ]
}

response = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers=headers,
    json=data
)

# ตรวจสอบและแสดงผลลัพธ์
response_json = response.json()
answer = response_json["choices"][0]["message"]["content"]
print("✅ คำตอบจาก GPT:", answer)


✅ คำตอบจาก GPT: จำนวนเงินสุทธิหลังภาษี ณที่จ่าย: 42,075.00

📌 [THAI VERSION]

ยอดรวมที่พิมพ์ไว้ในเอกสาร: 42,075.00

📌 [ENGLISH VERSION]

Total amount as printed in the document: 42,075.00
